In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
! wget https://download.pytorch.org/tutorial/data.zip

--2020-09-08 16:54:54--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 99.86.38.72, 99.86.38.37, 99.86.38.96, ...
Connecting to download.pytorch.org (download.pytorch.org)|99.86.38.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.05s   

2020-09-08 16:54:54 (56.9 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [3]:
! unzip data*

Archive:  data.zip
   creating: data/
  inflating: data/eng-fra.txt        
   creating: data/names/
  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0:'SOS',1:"EOS"}
    self.n_words = 2
  
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)
  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words +=1

    else: self.word2count[word] +=1

In [5]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
MAX_LENGTH = 10
eng_prefix = (
    'i am ', 'i m ',
    'he is', 'he s',
    'she is','she s '
    ,'you are', 'you re ',
    'we are','we re ',
    'they are','they re '
)

In [8]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefix)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData(lang1, lang2, reverse=False):
  input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
  print('Read %s sentence pairs'%len(pairs))
  pairs = filterPairs(pairs)
  print('Trimmed to %s sentence pairs' % len(pairs))
  print('Counting words...')
  for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
  print('counting words:')
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name , output_lang.n_words)
  return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng','fra',True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11066 sentence pairs
Counting words...
counting words:
fra 4601
eng 3019
['elle est affutee .', 'she is sharp .']


# encoder
![](https://pytorch.org/tutorials/_images/encoder-network.png)

In [10]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(EncoderRNN,self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
  
  def forward(self,input, hidden):
    embedded = self.embedding(input).view(1,1,-1)
    output = embedded
    output, hidden = self.gru(output, hidden)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size,device=device)
  

# decoder
![](https://pytorch.org/tutorials/_images/decoder-network.png)

In [11]:
class DecoderRNN(nn.Module):
  
  def __init__(self, hidden_size, output_size):
    super(DecoderRNN, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)
  
  def forward(self, input, hidden):
    output = self.embedding(input).view(1,1,-1)
    output = F.relu(output)
    output,hidden = self.gru(output,hidden)
    return output, hidden
  def initHidden(self):
    return torch.zeros(1,1, self.hidden_size, device=device)

# Attention decoder
![](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

In [12]:
class AttnDecoderRNN(nn.Module):

  def __init__(self, hidden_size, output_size, dropout_p=0.1,
               max_length = MAX_LENGTH):
    super(AttnDecoderRNN,self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout_p = dropout_p
    
    self.max_length = max_length
    self.embedding = nn.Embedding(self.output_size, self.hidden_size)
    self.attn = nn.Linear(self.hidden_size *2, self.max_length)
    
    self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
    self.dropout = nn.Dropout(self.dropout_p)
    
    self.gru = nn.GRU(self.hidden_size, self.hidden_size)
    self.out = nn.Linear(self.hidden_size, self.output_size)

  def forward(self, input, hidden, encoder_outputs):
    embedded = self.embedding(input).view(1,1,-1)
    embedded = self.dropout(embedded)

    attn_weights = F.softmax(
        self.attn(torch.cat((embedded[0], hidden[0]),1)),dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                             encoder_outputs.unsqueeze(0))
    output = torch.cat((embedded[0], attn_applied[0]),1)
    output = self.attn_combine(output).unsqueeze(0)

    output = F.relu(output)
    output, hidden = self.gru(output,hidden)
    output = F.log_softmax(self.out(output[0]),dim=1)
    return output,hidden,attn_weights

  def initHidden(self):
    return torch.zeros(1,1,self.hidden_size, device=device)


In [13]:
def indexesFromSentence(lang, sentence):
  return [lang.word2index[word] for word in sentence.split(" ")]

def tensorFromSentence(lang,sentence):
  indexes = indexesFromSentence(lang,sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes, dtype=torch.long,device=device).view(-1,1)

def tensorsFromPair(pair):
  input_tensor = tensorFromSentence(input_lang,pair[0])
  target_tensor = tensorFromSentence(output_lang, pair[1])
  return (input_tensor, target_tensor)


In [14]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [15]:
import time
import math

def asMinutes(s):
  m = math.floor(s/60)
  s -= m* 60
  return '%dm %ds' %(m,s)

def timeSince(since, percent):
  now = time.time()
  s = now - since
  es = s/(percent)
  rs = es-s
  return "%s (-%s)" %(asMinutes(s), asMinutes(rs))
  

In [16]:
def trainIters(encoder,decoder,n_iters,print_every=1000, plot_every=100, learning_rate=0.01):
  start = time.time()
  plot_losses = []
  print_loss_total  = 0 # reset every print_every
  plot_loss_total = 0 # reset every plot_every

  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(),lr=learning_rate)
  training_pairs = [tensorsFromPair(random.choice(pairs))
                    for i in range(n_iters)]
  criterion = nn.NLLLoss()

  for iter in range(1,n_iters + 1):
    training_pair = training_pairs[iter-1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]

    loss = train(input_tensor, target_tensor, encoder, decoder,
                 encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    plot_loss_total += loss

    if iter % print_every ==0:
      print_loss_avg = print_loss_total / print_every
      print_loss_total = 0
      print('%s (%d %d%%) %.4f' % (timeSince(start,iter/n_iters),
                                   iter, iter/n_iters * 100, print_loss_avg))
    if iter % plot_every == 0:
      plot_loss_avg = plot_loss_total / plot_every
      plot_losses.append(plot_loss_avg)
      plot_loss_total = 0


  showPlot(plot_losses)


In [17]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
  plt.figure()
  fig, ax = plt.subplots()
  loc = ticker.MultipleLocator(base=0.2)
  ax.yaxis.set_major_locator(loc)
  plt.plot(points)

In [20]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,dropout_p=0.1).to(device)
trainIters(encoder1,attn_decoder1,75000,print_every=5000)

1m 35s (-22m 21s) (5000 6%) 2.9154
3m 10s (-20m 35s) (10000 13%) 2.3518
4m 42s (-18m 48s) (15000 20%) 2.0462
6m 15s (-17m 13s) (20000 26%) 1.8300
7m 49s (-15m 39s) (25000 33%) 1.6475
9m 23s (-14m 5s) (30000 40%) 1.4664
10m 57s (-12m 31s) (35000 46%) 1.2865
12m 33s (-10m 59s) (40000 53%) 1.2040
14m 7s (-9m 25s) (45000 60%) 1.1033
15m 41s (-7m 50s) (50000 66%) 0.9843
17m 15s (-6m 16s) (55000 73%) 0.8949
18m 50s (-4m 42s) (60000 80%) 0.8257
20m 26s (-3m 8s) (65000 86%) 0.7659
22m 1s (-1m 34s) (70000 93%) 0.6830
23m 38s (-0m 0s) (75000 100%) 0.6536


In [21]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    for ei in range(input_length):
      encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                               encoder_hidden)
      decoder_input = torch.tensor([[SOS_token]], device=device)
      decoder_hidden = encoder_hidden 

      decoded_words = []
      decoder_attentions = torch.zeros(max_length, max_length)

      for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )

        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)

        if topi.item() == EOS_token:
          decoded_words.append("<EOS>")
          break
        else:
          decoded_words.append(output_lang.index2word[topi.item()])
        
        decoder_input = topi.squeeze().detach()
      return decoded_words, decoder_attentions[:di + 1]

In [22]:
def evaluateRandomly(encoder, decoder,n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [25]:
evaluateRandomly(encoder1, attn_decoder1)

> je fais tout mon possible .
= i m doing everything i can .
< . <EOS>

> c est un artiste celebre .
= he is a famous artist .
< i m open . <EOS>

> nous ne rajeunissons pas .
= we re not getting any younger .
< we re <EOS>

> il s assit pres d elle .
= he sat next to her .
< he s very sleepy . <EOS>

> vous etes l institutrice .
= you re the teacher .
< i m ready for you . <EOS>

> vous etes vraiment genial .
= you re really awesome .
< i m ready for you . <EOS>

> vous etes tellement previsible !
= you re so predictable .
< i m ready for you . <EOS>

> je ne suis pas encore prete .
= i am not ready yet .
< . <EOS>

> j ai tellement soif .
= i m so thirsty .
< i am . <EOS>

> tu es endurant .
= you re resilient .
< i m completely for . <EOS>

